In [132]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [133]:
from clipboard_health import main as ch
from clipboard_health import images as chi

import pandas as pd
from plotly import express as px

In [134]:
df = ch.load_base_data()

In [220]:
d = ['provname', 'provnum', 'city', 'state', 'workdate', 'county_name', 'county_fips', 
     'cy_qtr']
[x for x in df.columns if x not in ch.HOURS_COLS and x not in d and not x.startswith('hrs_')]

['mdscensus']

In [221]:
df['cy_qtr'].unique()

array(['2024Q2'], dtype=object)

In [144]:
df[['provname', 'provnum', 'workdate']].drop_duplicates().shape[0] / len(df)

1.0

In [145]:
df[['provname', 'provnum', 'workdate', 'city', 'state']].drop_duplicates().shape[0] / len(df)

1.0

In [146]:
len(df['provname'].unique())

14378

In [166]:
fp = '/l/tmp/NH_Ownership_Feb2025.csv'
d = pd.read_csv(fp)
d['ccn'] = d['CMS Certification Number (CCN)']
d['name'] = d['Owner Name']

In [168]:
d[['ccn', 'name']].drop_duplicates().shape[0] / len(d)

0.8206998998169067

In [165]:
print(d.columns)

Index(['CMS Certification Number (CCN)', 'Provider Name', 'Provider Address',
       'City/Town', 'State', 'ZIP Code',
       'Role played by Owner or Manager in Facility', 'Owner Type',
       'Owner Name', 'Ownership Percentage', 'Association Date', 'Location',
       'Processing Date', 'ccn'],
      dtype='object')


In [159]:
d['Owner Name'].unique().shape

(43270,)

In [169]:
d['ccn'].unique().shape

(14782,)

In [192]:
prid = df['provnum'].sample().iloc[0]

m = d['ccn'] == prid
m &= d['Role played by Owner or Manager in Facility'] == 'OPERATIONAL/MANAGERIAL CONTROL'
d[m]

,CMS Certification Number (CCN),Provider Name,Provider Address,City/Town,State,ZIP Code,Role played by Owner or Manager in Facility,Owner Type,Owner Name,Ownership Percentage,Association Date,Location,Processing Date,ccn,name


In [194]:
d[['Provider Name', 'ccn']].drop_duplicates().shape[0]

14782

In [195]:
d[['Owner Name', 'ccn']].drop_duplicates().shape[0]

118784

In [160]:
len(d)

144735

In [152]:
d['ccn'].dtype

dtype('O')

# Hours by Type

## fig = chi.hours_by_state(df)

# Hours by State and Type

In [81]:
fig = chi.hours_by_state(df, dev_path=True)

wrote '/l/tmp/hours-by-state.html'


In [82]:
chi.build_images(df)

wrote '/l/tmp/hours-by-role.html'
wrote '/l/tmp/hours-by-state.html'


# Hours by State and Contractor Ratio

In [206]:
d = chi.prep_ctr_ratio_hours(df, fancy=True)
d

,Total Hours,Employee Hours,Contractor Hours,Contractor / Total,Contractor Market Share
roletype,,,,,
Certified Nursing Assistant,"230,329K","214,185K","16,144K",7.0%,55.9%
Licensed Practical Nurse,"87,700K","79,571K","8,129K",9.3%,28.1%
Registered Nurse,"46,124K","42,397K","3,726K",8.1%,12.9%
RN with Admin Duties,"13,601K","13,269K",333K,2.4%,1.2%
Medication Aide,"11,369K","11,084K",285K,2.5%,1.0%
LPN with Admin Duties,"8,896K","8,806K",90K,1.0%,0.3%
RN Director of Nursing,"6,886K","6,758K",128K,1.9%,0.4%
Nurse Aide Trainee,"5,718K","5,649K",69K,1.2%,0.2%


In [7]:
df.columns

Index(['provnum', 'provname', 'city', 'state', 'county_name', 'county_fips',
       'cy_qtr', 'workdate', 'mdscensus', 'hrs_rndon', 'hrs_rndon_emp',
       'hrs_rndon_ctr', 'hrs_rnadmin', 'hrs_rnadmin_emp', 'hrs_rnadmin_ctr',
       'hrs_rn', 'hrs_rn_emp', 'hrs_rn_ctr', 'hrs_lpnadmin',
       'hrs_lpnadmin_emp', 'hrs_lpnadmin_ctr', 'hrs_lpn', 'hrs_lpn_emp',
       'hrs_lpn_ctr', 'hrs_cna', 'hrs_cna_emp', 'hrs_cna_ctr', 'hrs_natrn',
       'hrs_natrn_emp', 'hrs_natrn_ctr', 'hrs_medaide', 'hrs_medaide_emp',
       'hrs_medaide_ctr'],
      dtype='object')